In [1]:
# Standard imports
import numpy as np
import pandas as pd

# For web scraping
import requests
from bs4 import BeautifulSoup

# For performing regex operations
import re

import os
import datetime

from limitless_scrape import *
from limitless_analysis import *

In [2]:
# Get URL of most recent tournament
df_latenight = scrape_for_dates_and_url()
t_url = [df_latenight.iloc[0]['URL']]

In [3]:
# Create urls for this tourmanent
url_dict = create_urls(t_url)

In [4]:
# 6. Scrape urls in dict and add date
scrape_results_dict = multi_latenight_scrape(url_dict)
scrape_results_dict = add_date_to_dict(scrape_results_dict, df_latenight)

INFO:root:Scraping players table for https://play.limitlesstcg.com/tournament/ln68/standings


In [5]:
# Display Standings 
players_df = scrape_results_dict['https://play.limitlesstcg.com/tournament/ln68/']['players']
players_df.head()

,Place,Name,Country,Points,Record,Opp. Win %,Opp. Opp. %,Deck,List,Player ID
0,1,Juancho (PrimeTimeWizardTV),,40,13 - 3 - 1,,,Arceus Goodra,,auriz27
1,2,Alongwaytobepro,,38,12 - 3 - 2,,,Regis,,alongwaytobepro
2,3,Sanjay7s,,40,13 - 2 - 1,,,Lost Zone Box,,sanjay7s
3,4,UltraNko,,34,11 - 4 - 1,,,Giratina LZ Box,,ultranko
4,5,David Hendrickson,,34,11 - 3 - 1,,,Palkia Inteleon,,royalcheez


In [6]:
# Display Pairings 
round_dict = scrape_results_dict['https://play.limitlesstcg.com/tournament/ln68/']['pairings']['round_1_dict']
round1_df = scrape_results_dict['https://play.limitlesstcg.com/tournament/ln68/']['pairings']['round_1_dict']['df']
round1_df.head()

,Pairing,Player 1,Player 1 Score,Player 2 Score,Player 2,Player 1 Name,Player 2 Name,Player 1 Record,Player 2 Record,Winner ID,Player 1 ID,Player 2 ID
0,1,Timm1011-0-0,0,0,inosoycarly0-1-0,Timm101,inosoycarly,1-0-0,0-1-0,timm101,timm101,inosoycarly
1,2,mryeet123412340-1-0,0,0,Orion Sutherland1-0-0,mryeet12341234,Orion Sutherland,0-1-0,1-0-0,joshuasutherland21,mryeet12341234,joshuasutherland21
2,3,Taylor Metz0-1-0,0,0,Mwaubin1-0-0,Taylor Metz,Mwaubin,0-1-0,1-0-0,mwaubin,zoura,mwaubin
3,4,grantm121-0-0,1,0,amcharles10-1-0,grantm12,amcharles1,1-0-0,0-1-0,grantm12,grantm12,amcharles1
4,5,compYammu0-1-0,0,1,Terryharris78991-0-0,compYammu,Terryharris7899,0-1-0,1-0-0,terryharris7899,compyammu,terryharris7899


In [7]:
deck_and_records(round_dict, players_df).head()

,Pairing,Player 1,Player 1 Score,Player 2 Score,Player 2,Player 1 Name,Player 2 Name,Player 1 Record,Player 2 Record,Winner ID,Player 1 ID,Player 2 ID,Player 1 Deck,Player 2 Deck,Winning Deck
0,1,Hisuian Zoroark,0,0,Regis,Timm101,inosoycarly,1-0-0,0-1-0,timm101,timm101,inosoycarly,Hisuian Zoroark,Regis,Hisuian Zoroark
1,2,Calyrex Inteleon,0,0,Arceus,mryeet12341234,Orion Sutherland,0-1-0,1-0-0,joshuasutherland21,mryeet12341234,joshuasutherland21,Calyrex Inteleon,Arceus,Arceus
2,3,Kyurem Palkia,0,0,Blissey,Taylor Metz,Mwaubin,0-1-0,1-0-0,mwaubin,zoura,mwaubin,Kyurem Palkia,Blissey,Blissey
3,4,Kyurem Palkia,1,0,Hisuian Zoroark,grantm12,amcharles1,1-0-0,0-1-0,grantm12,grantm12,amcharles1,Kyurem Palkia,Hisuian Zoroark,Kyurem Palkia
4,5,Arceus Giratina,0,1,Arceus Goodra,compYammu,Terryharris7899,0-1-0,1-0-0,terryharris7899,compyammu,terryharris7899,Arceus Giratina,Arceus Goodra,Arceus Goodra


In [8]:
# 7. Process data: Get WLT counts for each deck in each tournament
all_tournament_results_dict = multi_tournament_wr_per_tournament(scrape_results_dict)

# Calculate winrates
multi_tournament_wr_calc(all_tournament_results_dict)

# Create plot_df
plot_df = create_plot_df(all_tournament_results_dict)

In [12]:
# Attempt to make heatmap
groupby_df = plot_df.groupby(['deck', 'opposing_deck']).agg({"winrate": "mean", "games_played": "sum"}).reset_index()

In [30]:
# All archetypes in 'deck' columns
all_archetypes = [x for x in groupby_df['deck'].unique()]

# Add archetypes in 'opposing_deck'
opp_decks = [x for x in groupby_df['opposing_deck'].unique()]

for deck in [archetype for archetype in opp_decks if archetype not in all_archetypes]:
    all_archetypes.append(deck)

In [33]:
# Create heatmap_df
heatmap_df = pd.DataFrame(columns=all_archetypes, index=all_archetypes)

In [34]:
for deck in heatmap_df:
    # get win rate 
    filtered_df = groupby_df[groupby_df['deck'] == deck]
    

Arcanine Zoroark
Arceus
Arceus Duraludon
Arceus Flying Pikachu
Arceus Giratina
Arceus Goodra
Arceus Mewtwo
Arceus Rayquaza
Blissey
Calyrex Inteleon
Charizard
Cramorant
Crobat Weezing
Dialga
Dialga Bronzong
Dialga LZ
Durant
Eternatus
Flaaffy
Galarian Weezing
Gengar
Giratina LZ Box
Hisuian Goodra
Hisuian Samurott
Hisuian Zoroark
Ice Rider Calyrex
Inteleon Box
Kyurem
Kyurem Palkia
Lost Zone Box
Lunatone Solrock
Machamp
Mew Genesect
Other
Palkia Inteleon
Rapid Strike Urshifu
Regis
Shadow Rider Calyrex
Suicune Inteleon
Urshifu Inteleon
Vikavolt
Volcarona
Zeraora


In [50]:
filtered_df = groupby_df[groupby_df['deck'] == "Palkia Inteleon"]

In [51]:
filtered_df

,deck,opposing_deck,winrate,games_played
368,Palkia Inteleon,Arcanine Zoroark,0.00,3
369,Palkia Inteleon,Arceus,0.75,4
370,Palkia Inteleon,Arceus Duraludon,0.50,2
371,Palkia Inteleon,Arceus Flying Pikachu,0.00,1
372,Palkia Inteleon,Arceus Giratina,1.00,2
373,Palkia Inteleon,Arceus Goodra,0.40,15
374,Palkia Inteleon,Arceus Mewtwo,1.00,1
375,Palkia Inteleon,Blissey,0.12,8
376,Palkia Inteleon,Crobat Weezing,1.00,1
377,Palkia Inteleon,Dialga,0.25,4


In [56]:
# Iterate through each active deck
for active_deck in heatmap_df:
    filtered_df = groupby_df[groupby_df['deck']==active_deck]
    
    # Find win rate for every deck in index (opposing deck)
    mu = []
    for deck in heatmap_df.index:
        if deck in filtered_df['opposing_deck'].unique():
            winrate = filtered_df[filtered_df['opposing_deck']==deck].iloc[0]['winrate']
            mu.append(winrate)
        else:
            winrate = None
            mu.append(winrate)
        
    # set column to mu 
    heatmap_df[active_deck] = mu

In [57]:
heatmap_df

,Arcanine Zoroark,Arceus,Arceus Duraludon,Arceus Flying Pikachu,Arceus Giratina,Arceus Goodra,Arceus Mewtwo,Arceus Rayquaza,Blissey,Calyrex Inteleon,...,Other,Palkia Inteleon,Rapid Strike Urshifu,Regis,Shadow Rider Calyrex,Suicune Inteleon,Urshifu Inteleon,Vikavolt,Volcarona,Zeraora
Arcanine Zoroark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,...,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
Arceus,NaN,0.50,NaN,NaN,0.00,0.50,NaN,NaN,0.67,0.0,...,NaN,0.75,NaN,0.33,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Duraludon,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,...,0.00,0.50,NaN,1.00,1.00,NaN,NaN,1.0,NaN,NaN
Arceus Flying Pikachu,NaN,NaN,NaN,NaN,NaN,0.67,NaN,NaN,NaN,NaN,...,0.00,0.00,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Giratina,NaN,1.00,NaN,NaN,NaN,1.00,NaN,0.0,0.33,NaN,...,0.00,1.00,NaN,0.12,NaN,NaN,NaN,0.0,NaN,NaN
Arceus Goodra,NaN,0.50,0.67,0.0,0.00,0.40,1.00,NaN,0.50,0.0,...,0.33,0.40,NaN,0.30,0.75,NaN,NaN,1.0,0.5,NaN
Arceus Mewtwo,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.67,NaN,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Rayquaza,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,0.00,NaN,...,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
Blissey,1.00,0.00,0.67,NaN,0.67,0.10,0.33,1.0,0.50,NaN,...,0.00,0.12,NaN,0.67,1.00,NaN,NaN,NaN,NaN,NaN
Calyrex Inteleon,NaN,1.00,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
len(mu)

43

In [52]:
# Find win rate for every deck in index (opposing deck)
mu = []
for deck in heatmap_df.index:
    if deck in filtered_df['opposing_deck'].unique():
        winrate = filtered_df[filtered_df['opposing_deck']==deck].iloc[0]['winrate']
        mu.append(winrate)
    else:
        winrate = None
        mu.append(winrate)

In [53]:
len(mu)

43

In [54]:
heatmap_df['Palkia Inteleon'] = mu

In [55]:
heatmap_df

,Arcanine Zoroark,Arceus,Arceus Duraludon,Arceus Flying Pikachu,Arceus Giratina,Arceus Goodra,Arceus Mewtwo,Arceus Rayquaza,Blissey,Calyrex Inteleon,...,Other,Palkia Inteleon,Rapid Strike Urshifu,Regis,Shadow Rider Calyrex,Suicune Inteleon,Urshifu Inteleon,Vikavolt,Volcarona,Zeraora
Arcanine Zoroark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Duraludon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Flying Pikachu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Giratina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Goodra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Mewtwo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arceus Rayquaza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Blissey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Calyrex Inteleon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# 1. Filter data on active deck 
# 2. Filter data based on archetypes in opposing decks
# 3. Create a blank heatmap with columns/indices for each archetype in filtered dataset 
# 4. Group filtered dataframe to get mean win rate and total games played 
# 5. Get win rates by looping through list of columns 
# 6. Plot the heatmap 

# 1
active_deck = 'Palkia Inteleon'
opposing_decks = ["Palkia Inteleon", "Kyurem Palkia", "Lost Zone Box", "Giratina LZ Box", "Mew Genesect"]
opposing_decks.append(active_deck)
unique_decks = []
for deck in opposing_decks:
    if deck not in unique_decks:
        unique_decks.append(deck)

# 2
active_deck_df = plot_df[plot_df["deck"].isin(unique_decks)]
filtered_df = active_deck_df[active_deck_df["opposing_deck"].isin(unique_decks)]

# 3 
opposing_decks.append(active_deck)
unique_decks = []
for deck in opposing_decks:
    if deck not in unique_decks:
        unique_decks.append(deck)
        
# blank heatmaps for wins and games played
heatmap_gp = pd.DataFrame(columns=unique_decks, index=unique_decks)
heatmap_wr = pd.DataFrame(columns=unique_decks, index=unique_decks)

# Create aggregated df 
groupby_df = filtered_df.groupby(['deck', 'opposing_deck']).agg({"winrate": "mean", "games_played": "sum"}).reset_index()

# Iterate through each active deck
for active_deck in heatmap_wr:
    filtered_df = groupby_df[groupby_df['deck']==active_deck]
    
    # Find win rate for every deck in index (opposing deck)
    wr_ls = []
    gp_ls = []
    for deck in heatmap_wr.index:
        if deck in filtered_df['opposing_deck'].unique():
            winrate = filtered_df[filtered_df['opposing_deck']==deck].iloc[0]['winrate']
            gp = filtered_df[filtered_df['opposing_deck']==deck].iloc[0]['games_played']
            wr_ls.append(winrate)
            gp_ls.append(gp)
        else:
            winrate = None
            mu.append(winrate)
            gp_ls.append(None)
        
    # set column to mu 
    heatmap_wr[active_deck] = wr_ls
    heatmap_gp[active_deck] = gp_ls


In [130]:
# heatmap hovertext; active deck, opposing deck, win rate, games played
hovertext = []
for xi, xx in enumerate(heatmap_wr.columns):
    hovertext.append(list())
    for yi, yy in enumerate(heatmap_wr.index):
        hovertext[-1].append(f"Active Deck: {xx}<br />Opposing Deck: {yy}<br />Win Rate: {heatmap_wr.values[yi][xi]}<br />Games Played: {heatmap_gp.values[yi][xi]}")

In [133]:


# construct heatmap 
import plotly.graph_objects as go 
fig = go.Figure(data=go.Heatmap(
                   z=heatmap_wr.values,
                   x=heatmap_wr.columns.tolist(),
                   y=heatmap_wr.index.tolist(),
                   hoverinfo='text',
                   text=hovertext,
                   texttemplate="%{z}",
                   textfont={"size": 20},
                   hoverongaps = False,
                   xgap=3, 
                   ygap=3
))
fig.show()

In [102]:
heatmap_wr

,Palkia Inteleon,Kyurem Palkia,Lost Zone Box,Giratina LZ Box,Mew Genesect
Palkia Inteleon,0.47,0.10,0.57,0.50,0.50
Kyurem Palkia,0.90,0.50,0.48,0.42,0.47
Lost Zone Box,0.43,0.48,0.48,0.42,0.52
Giratina LZ Box,0.50,0.53,0.53,0.47,0.37
Mew Genesect,0.50,0.47,0.44,0.58,0.48


In [105]:
heatmap_wr.values

array([[0.47, 0.1 , 0.57, 0.5 , 0.5 ],
       [0.9 , 0.5 , 0.48, 0.42, 0.47],
       [0.43, 0.48, 0.48, 0.42, 0.52],
       [0.5 , 0.53, 0.53, 0.47, 0.37],
       [0.5 , 0.47, 0.44, 0.58, 0.48]])

In [100]:
plot_df[(plot_df['deck']=="Palkia Inteleon") & (plot_df['opposing_deck']=="Kyurem Palkia")]

,deck,opposing_deck,t_url,date,winrate,games_played
124,Palkia Inteleon,Kyurem Palkia,https://play.limitlesstcg.com/tournament/ln68/,2022-11-01,0.9,10


In [113]:
test = list()

In [114]:
for xi,xx in enumerate(range(10)):
    test.append(list())
    for yi, yy in enumerate(range(10)):
        test[-1].append(f"{xi, yi}")

In [126]:
hovertext = []
for xi, xx in enumerate(heatmap_wr.columns):
    hovertext.append(list())
    for yi, yy in enumerate(heatmap_wr.index):
        hovertext[-1].append(f"Active Deck: {xx}<br />Opposing Deck: {yy}<br />Win Rate: {heatmap_wr.values[yi][xi]}<br />Games Played: {heatmap_gp.values[yi][xi]}")

In [127]:
hovertext

[['Active Deck: Palkia Inteleon<br />Opposing Deck: Palkia Inteleon<br />Win Rate: 0.47<br />Games Played: 34',
  'Active Deck: Palkia Inteleon<br />Opposing Deck: Kyurem Palkia<br />Win Rate: 0.9<br />Games Played: 10',
  'Active Deck: Palkia Inteleon<br />Opposing Deck: Lost Zone Box<br />Win Rate: 0.43<br />Games Played: 14',
  'Active Deck: Palkia Inteleon<br />Opposing Deck: Giratina LZ Box<br />Win Rate: 0.5<br />Games Played: 38',
  'Active Deck: Palkia Inteleon<br />Opposing Deck: Mew Genesect<br />Win Rate: 0.5<br />Games Played: 18'],
 ['Active Deck: Kyurem Palkia<br />Opposing Deck: Palkia Inteleon<br />Win Rate: 0.1<br />Games Played: 10',
  'Active Deck: Kyurem Palkia<br />Opposing Deck: Kyurem Palkia<br />Win Rate: 0.5<br />Games Played: 12',
  'Active Deck: Kyurem Palkia<br />Opposing Deck: Lost Zone Box<br />Win Rate: 0.48<br />Games Played: 25',
  'Active Deck: Kyurem Palkia<br />Opposing Deck: Giratina LZ Box<br />Win Rate: 0.53<br />Games Played: 19',
  'Active Deck:

In [123]:
heatmap_wr.values

array([[0.47, 0.1 , 0.57, 0.5 , 0.5 ],
       [0.9 , 0.5 , 0.48, 0.42, 0.47],
       [0.43, 0.48, 0.48, 0.42, 0.52],
       [0.5 , 0.53, 0.53, 0.47, 0.37],
       [0.5 , 0.47, 0.44, 0.58, 0.48]])